<a href="https://colab.research.google.com/github/avii09/semantic-search-engine/blob/main/search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install streamlit
!npm install -g localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.184s


In [ ]:
%%writefile app.py
import streamlit as st
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import gzip
import os
import torch

Overwriting app.py


In [ ]:
model_name ='nq-distilbert-base-v1'
model = SentenceTransformer(model_name)
#this is the model used for generating vectors (encoding)(biencoder)

In [ ]:
top_k = 5
#the no. of passages we want

In [ ]:
text = r"/content/simplewiki-2020-11-01.jsonl"
# this is the corpus data which will be put into the biencoder to generate the corpus embeddings.


#here we are splitting the dataset as paragraphs and encode them.
corpus = []
with open(text, "rt", encoding="utf8") as fIn:
    for line in fIn:
        data = json.loads(line.strip())
        for paragraph in data["paragraphs"]:
            corpus.append([data["title"], paragraph])     #we split them as [title, paragraphs]

print("corpus:", len(corpus))


#here we are generating the embeddings for the split corpus dataset. This is our corpus dataset.
if model_name == 'nq-distilbert-base-v1':
    embeddings_filepath = 'simplewiki-2020-11-01-nq-distilbert-base-v1.pt'
    if not os.path.exists(embeddings_filepath):
        util.http_get('http://sbert.net/datasets/simplewiki-2020-11-01-nq-distilbert-base-v1.pt', embeddings_filepath)

    corpus_embeddings = torch.load(embeddings_filepath, map_location=torch.device('cpu'))
    corpus_embeddings = corpus_embeddings.float()  # Convert embedding file to float
    if torch.cuda.is_available():
        corpus_embeddings = corpus_embeddings.to('cuda')

corpus: 509663


In [ ]:
%%writefile app.py
#now we ask for the query and embed the query.
def search(query):
    # Encode the query using the bi-encoder and find potentially relevant passages
    start_time = time.time()
    query_embedding = model.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query
    end_time = time.time()

    sorted_hits = sorted(hits, key=lambda x: x['score'])

    # Output of top-k hits
    results = []
    for hit in sorted_hits:
      title, paragraph = corpus[hit['corpus_id']]
      results.append((title, paragraph))
    return results
    print("Results (after {:.3f} seconds):".format(end_time - start_time))



import streamlit as st
st.title('Semantic Search Engine')
query = st.text_input('Enter your query:')
if query:
    results = search(query)
    st.write(f"Input question: {query}")
    st.write("Results:")
    for title, paragraph in results:
        st.write(f"**Title:** {title}")
        st.write(f"**Paragraph:** {paragraph}")
        st.write("\n")

# Streamlit UI

Overwriting app.py


In [ ]:
%%writefile app.py
import streamlit as st
st.title('Semantic Search Engine')
query = st.text_input('Enter your query:')
if query:
    results = search(query)
    st.write(f"Input question: {query}")
    st.write("Results:")
    for title, paragraph in results:
        st.write(f"**Title:** {title}")
        st.write(f"**Paragraph:** {paragraph}")
        st.write("\n")

# Streamlit UI



Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.96.151:8501

npx: installed 22 in 2.793s
your url is: https://calm-files-go.loca.lt
2024-07-03 11:34:03.633 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 589, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 26, in <module>
    results = search(query)
  File "/content/app.py", line 4, in search
    start_time = time.time()
NameError: name 'time' is not defined
  Stopping...
^C


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.80.79.204


In [ ]:
search(query = "What is the capital of the France?")

Input question: What is the capital of the France?
Results (after 4.335 seconds):
	0.745	['Arrondissement of Confolens', 'The arrondissement of Confolens is an arrondissement of France, in the Charente department, Nouvelle-Aquitaine region. Its capital is the city of Confolens.']
	0.746	["Arrondissement of Saint-Jean-d'Angély", "The arrondissement of Saint-Jean-d'Angély is an arrondissement of France, in the Charente-Maritime department, Nouvelle-Aquitaine region. Its capital is the city of Saint-Jean-d'Angély."]
	0.752	['Arrondissement of Figeac', 'The arrondissement of Figeac is an arrondissement of France. It is part of the Lot "département" in the Occitanie region. Its capital is the city of Figeac.']
	0.753	['Arrondissement of Sarlat-la-Canéda', 'The arrondissement of Sarlat-la-Canéda is an arrondissement of France. It is part of the Dordogne "département" in the Nouvelle-Aquitaine region. Its capital is the city of Sarlat-la-Canéda.']
	0.826	['Capital of France', 'The capital of 

In [ ]:
search(query="who turned out to be the mother on how i met your mother")

Input question: who turned out to be the mother on how i met your mother
Results (after 1.946 seconds):
	0.513	['Josh Radnor', 'Joshua Thomas Radnor (born July 29, 1974) is an American actor, director, producer, and screenwriter. He is best known for playing Ted Mosby on the CBS sitcom "How I Met Your Mother".']
	0.583	['Jason Segel', 'Jason Jordan Segel (born January 18, 1980) is an American movie, television, and voice actor who is known for his role as Marshall Eriksen in "How I Met Your Mother". He has also appeared in "The Muppets" (2011). He was born in Los Angeles, California. He was partners with actress Michelle Williams from 2012 through early 2013.']
	0.597	['How I Met Your Mother', 'The story is set in New York City. It is about a man named Ted Mosby who tells his son and daughter how he met their mother. Ted is played by Josh Radnor and the story is narrated by Bob Saget who plays Ted Mosby in the future year 2030. Other characters and friends of Ted are Marshall Eriksen (

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.80.79.204
